In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Read data, index by date

In [0]:
fileurl = "https://s3-eu-west-1.amazonaws.com/training180529/data/djia_close.csv"
df = pd.read_csv(fileurl, sep=',',header=0)
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by= 'date').reset_index(drop=True)

### 1. Validation as random samples in the timeline

In [0]:
train=df.sample(frac=2/3,random_state=200)
test=df.drop(train.index)

In [0]:
print(train.shape)
import math

In [0]:
print(train.shape)
import math

In [0]:
learning_rate = 2e-8
training_steps = 50000
losses = []
errors = []

with tf.Session() as sess:
    x = tf.constant(train.drop(columns=['date', 'JPM', 'DWDP', 'MMM']))
#    x = tf.constant(train[['AXP']])
    y = tf.constant(train[['JPM']])
    
    x_test = tf.constant(test.drop(columns=['date', 'JPM', 'DWDP', 'MMM']))
    y_test = tf.constant(test[['JPM']])
    
    weights = tf.Variable(tf.random_normal([27, 1], 0, 1.0/30, dtype=tf.float64))
    
    b = tf.Variable(tf.random_normal([1], 0, 0.1, dtype=tf.float64))
    
    tf.global_variables_initializer().run()
    yhat = x @ weights + b
    yerror = tf.subtract(y,yhat)
    
    loss = tf.nn.l2_loss(yerror)
    update_weights = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    yerror_test = x_test @ weights + b - y_test
    loss_test = tf.nn.l2_loss(yerror_test)
    
    for _ in range(training_steps):
        # Repeatedly run the operations, updating the TensorFlow variable.
        sess.run(update_weights)
        losses.append(math.sqrt(loss.eval()/train.shape[0]))
        errors.append(math.sqrt(loss_test.eval()/test.shape[0]))
        #print("{} vs {}".format(loss_test.eval(), loss.eval()))
    betas = weights.eval()
    bias = b.eval()
    yhat = yhat.eval()
    y = y.eval()


In [0]:
plt.yscale('log')
plt.plot(range(0, training_steps), losses)
plt.plot(range(0, training_steps), errors)

In [0]:
print(errors[-1])

### 2. Validation as 3rd third of data in the timesline order

In [0]:
split_idx = int(2*df.shape[0]/3)

In [0]:
train=df[:split_idx]#.sample(frac=0.8,random_state=200)
test=df[split_idx:]#.drop(train.index)

In [0]:
learning_rate = 2e-8
training_steps = 50000
losses_1 = []
errors_1 = []

with tf.Session() as sess:
    x = tf.constant(train.drop(columns=['date', 'JPM', 'DWDP', 'MMM']))
#    x = tf.constant(train[['AXP']])
    y = tf.constant(train[['JPM']])
    
    x_test = tf.constant(test.drop(columns=['date', 'JPM', 'DWDP', 'MMM']))
    y_test = tf.constant(test[['JPM']])
    
    weights = tf.Variable(tf.random_normal([27, 1], 0, 1.0/30, dtype=tf.float64))
    
    b = tf.Variable(tf.random_normal([1], 0, 0.1, dtype=tf.float64))
    
    tf.global_variables_initializer().run()
    yhat = x @ weights + b
    yerror = tf.subtract(yhat, y)
    
    loss = tf.nn.l2_loss(yerror)
    update_weights = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    yerror_test = x_test @ weights + b - y_test
    loss_test = tf.nn.l2_loss(yerror_test)
    
    for _ in range(training_steps):
        # Repeatedly run the operations, updating the TensorFlow variable.
        sess.run(update_weights)
        losses_1.append(math.sqrt(loss.eval()/train.shape[0]))
        errors_1.append(math.sqrt(loss_test.eval()/test.shape[0]))
        #print("{} vs {}".format(loss_test.eval(), loss.eval()))
    betas = weights.eval()
    bias = b.eval()
    yhat = yhat.eval()
    y = y.eval()


Plot RMSE for both validation schemes, see how time-ordered validation proves less prone to overfitting as RMSE remains higher than pure random-picked validation samples

In [0]:
plt.yscale('log')
plt.plot(range(0, training_steps), losses, '#0000FF')
plt.plot(range(0, training_steps), errors, '#00FFFF')
plt.plot(range(0, training_steps), losses_1, '#FF0000')
plt.plot(range(0, training_steps), errors_1, '#FFFF00')

In [0]:
print(np.min(losses))
print(np.min(losses_1))
print(np.min(errors))
print(np.min(errors_1))